In [2]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina' # I use a HiDPI screen

In [3]:
from fishchips.experiments import CMB_Primary
from fishchips.cosmo import Observables
import fishchips.cmb_lensing
import fishchips.util

from classy import Class  # CLASS python wrapper
import numpy as np
import matplotlib.pyplot as plt
default_color_list = ['#1A1A1A', '#0072B2', '#009E73', '#D55E00', '#CC79A7', '#F0E442', '#56B4E9']

I've pickled a few fishchips experiment objects (Planck, S3, S4) so you don't have to install orphics. We load it in, and get a dictionary with the three experiments.

In [4]:
import pickle
premades = pickle.load( (open("../data/premades.p", "rb")) )

premades

{'Planck': [<fishchips.experiments.CMB_Primary at 0x7fa89109e358>,
 'S3': [<fishchips.experiments.CMB_Primary at 0x7fa8911228d0>,
 'S4': [<fishchips.experiments.CMB_Primary at 0x7fa8911c3b70>,
  <fishchips.cmb_lensing.CMB_Lensing_Only at 0x7fa8911c5f98>]}

Next let's generate an observables object.

In [5]:
# create an Observables object to store information for derivatives
pars = np.array( ['omega_b', 'omega_cdm', 'h',  'A_s', 'n_s', 'tau_reio'])
centers = np.array([0.02222,  0.1197,  0.69,  2.1955e-9, 0.9655, 0.06])
steps = np.array([0.0001,     0.001,    0.01,   0.01e-9, 0.005, 0.02])

obs = Observables(parameters=pars,
                  fiducial=centers,
                  left=centers-steps,
                  right=centers+steps)

# generate a template CLASS python wrapper configuration
classy_template = {'output': 'tCl pCl lCl',
                   'l_max_scalars': 5000,
                   'lensing': 'yes',
                   'non linear': 'halofit'}
# add in the fiducial values too
classy_template.update(dict(zip(obs.parameters, obs.fiducial)))

# generate the fiducial cosmology
obs.compute_cosmo(key='fiducial', classy_dict=classy_template)

# generate an observables dictionary, looping over parameters
for par, par_left, par_right in zip(obs.parameters, obs.left, obs.right):
    classy_left = classy_template.copy()
    classy_left[par] = par_left
    classy_right = classy_template.copy()
    classy_right[par] = par_right
    # pass the dictionaries full of configurations to get computed
    obs.compute_cosmo(key=par + '_left', classy_dict=classy_left)
    obs.compute_cosmo(key=par + '_right', classy_dict=classy_right)

Let's compute a Fisher matrix using the premade S4!

In [6]:
# sum together fisher matrices
fisher = np.sum( [exp.get_fisher(obs, lensed_Cl=True)
                  for exp in premades["S4"]], axis=0 )
cov = np.linalg.inv(fisher) # invert result to get covariance on parameters

cov
#fishchips.util.plot_triangle(obs, cov, 
 #                            labels=[r"$\Omega_b h^2$", r"$\Omega_c h^2$", r"$h$",
  #                                  r"$A_s$", r"$n_s$", r"$\tau_{reio}$"]);

array([[ 8.73287111e-10,  1.33690123e-09,  2.37168419e-09,
         1.52566484e-17, -3.41092592e-08, -5.60746230e-09],
       [ 1.33690123e-09,  3.39114700e-07, -1.27600796e-06,
        -1.17505806e-14, -8.89498986e-07, -3.20886263e-06],
       [ 2.37168419e-09, -1.27600796e-06,  4.94000850e-06,
         4.48439055e-14,  3.16857646e-06,  1.21591983e-05],
       [ 1.52566484e-17, -1.17505806e-14,  4.48439055e-14,
         4.37841407e-22,  2.69544471e-14,  1.17018743e-13],
       [-3.41092592e-08, -8.89498986e-07,  3.16857646e-06,
         2.69544471e-14,  4.61249025e-06,  8.34002986e-06],
       [-5.60746230e-09, -3.20886263e-06,  1.21591983e-05,
         1.17018743e-13,  8.34002986e-06,  3.18287417e-05]])

The premades pickled dictionary was made with the following code:

In [9]:
# import pickle

# p = fishchips.experiments.get_PlanckPol_combine()
# s3 = fishchips.experiments.get_S3() + [fishchips.experiments.get_S3_Lensing_Only()]
# s4 = fishchips.experiments.get_S4() + [fishchips.experiments.get_S4_Lensing_Only()]
# print("Planck\n", p)
# print("S3\n", s3)
# print("S4\n", s4)

# import pickle
# pickle.dump({"Planck": p, "S3":s3, "S4":s4},  open( "../data/premades.p", "wb") )